In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import os
import glob
import itertools

In [2]:
path = "Datasets/"

In [95]:
def getProtectedAttributesTwitch(G, df):
    protected_attributes = dict(zip(df["numeric_id"], df["mature"]))
    newG = nx.set_node_attributes(G, protected_attributes, "mature")
    return newG

def getProtectedAttributesDeezer(G, df):
    protected_attributes = dict(zip(df["id"], df["target"]))
    newG = nx.set_node_attributes(G, protected_attributes, "gender")
    return newG

def getProtectedAttributesPokec(G,df):
    protected_attributes = dict(zip(df[0], df[3]))
    newG = nx.set_node_attributes(G, protected_attributes, "gender")
    return newG

def getprotectedAttributesDict_Facebook(featuresDF,featureNameDF,egoFeatDF):
    gender_index = featureNameDF.index[featureNameDF[1] == "gender;anonymized"].to_list()[0]
    featuresDF = featuresDF[[0, gender_index + 1]]
    egoFeatDF = egoFeatDF[[0, gender_index + 1]]
    featuresDict = dict(zip(featuresDF[0], featuresDF[gender_index + 1]))
    egoFeatDict = dict(zip(egoFeatDF[0], egoFeatDF[gender_index + 1]))
    featuresDict.update(egoFeatDict)
    return featuresDict

def getprotectedAttributesDict_GPlus(featuresDF, gender_index, egoFeatDF):
    # gender_index = featureNameDF.index[featureNameDF[1] == "gender:1"].to_list()[0]
    featuresDF = featuresDF[[0, gender_index + 1]]
    egoFeatDF = egoFeatDF[[0, gender_index + 1]]
    featuresDict = dict(zip(featuresDF[0], featuresDF[gender_index + 1]))
    egoFeatDict = dict(zip(egoFeatDF[0], egoFeatDF[gender_index + 1]))
    featuresDict.update(egoFeatDict)
    return featuresDict

def getEgoFeats(path):
    featFiles = []
    featNameFiles = []
    egoFeatFiles = []
    for file in os.listdir(path):
        if file.endswith(".feat"):
            featFiles.append(file)
        if file.endswith(".featnames"):
            featNameFiles.append(file)
        if file.endswith(".egofeat"):
            egoFeatFiles.append(file)
    return featFiles, featNameFiles, egoFeatFiles

def getProtectedAttributesFacebook(G):
    node_gender_dict = {}
    localpath = "Datasets/facebook/facebook/"
    fbFeatFiles, fbFeatNameFiles, fbEgoFeatFiles = getEgoFeats(localpath)
    for index in range(len(fbFeatFiles)):
        localFeaturesDF = pd.read_csv(localpath + fbFeatFiles[index], sep=" ", header=None)
        localFeatureNamesDf = pd.read_csv(localpath + fbFeatNameFiles[index], sep=" ", header=None)
        localEgoFeatDf = pd.read_csv(localpath + fbEgoFeatFiles[index], sep=" ", header=None)
        protectedAttrDict = getprotectedAttributesDict_Facebook(localFeaturesDF, localFeatureNamesDf, localEgoFeatDf)
        node_gender_dict.update(protectedAttrDict)
    
    newG = nx.set_node_attributes(G, node_gender_dict, "gender")
    return newG

def getProtectedAttributesGPlus(G):
    node_gender_dict = {}
    localpath = "Datasets/gplus/"
    gplusFeatFiles, gplusFeatNameFiles, gplusEgoFeatFiles = getEgoFeats(localpath)
    for index in range(len(gplusFeatFiles)):
        try:
            localFeaturesDF = pd.read_csv(localpath + gplusFeatFiles[index], sep=" ", header=None)
        # localFeatureNamesDf = pd.read_csv(localpath + gplusFeatNameFiles[index], sep=" ", header=None)
            localEgoFeatDf = pd.read_csv(localpath + gplusEgoFeatFiles[index], sep=" ", header=None)
            protectedAttrDict = getprotectedAttributesDict_GPlus(localFeaturesDF, 0, localEgoFeatDf)
            node_gender_dict.update(protectedAttrDict)
        except:
            print(gplusEgoFeatFiles[index])
    
    print(node_gender_dict)
    newG = nx.set_node_attributes(G, node_gender_dict, "gender")
    return newG


In [84]:
localpath = "Datasets/gplus/"
gplusFeatFiles, gplusFeatNameFiles, gplusEgoFeatFiles = getEgoFeats(localpath)
localEgoFeatDf = pd.read_csv(localpath + gplusEgoFeatFiles[0], sep=" ", header=None)
# gplusEgoFeatFiles[0]
localEgoFeatDf
# gplusFeatNameFiles[0]
# test = pd.read_csv("Datasets/gplus/100129275726588145876.featnames")

,0,1,2,3,4,5,6,7,8,9,...,1309,1310,1311,1312,1313,1314,1315,1316,1317,1318
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
def initialize_community_attribute_Counter(communitiesList):
    protectedAttributeCountDict = {}
    communityCount = 0
    for i in range(len(communitiesList)):
        protectedAttributeCountDict["Community_{}".format(communityCount)] = {0: 0, 1: 0}
        communityCount += 1
    return protectedAttributeCountDict

def count_protected_attributes_frequency(G, communitiesList, protectedAttributeCountDict, attribute):
    communityCount = 0
    for community in communitiesList:
        for node in community:
            try:
                if G.nodes()[node][attribute] == 0:
                    protectedAttributeCountDict["Community_{}".format(communityCount)][0] += 1
                else:
                    protectedAttributeCountDict["Community_{}".format(communityCount)][1] += 1
            except KeyError:
                continue
        communityCount += 1
    return protectedAttributeCountDict

def calculate_community_balance(protectedAttributeCountDict):
    for x in protectedAttributeCountDict:
        red = protectedAttributeCountDict[x][0]
        blue = protectedAttributeCountDict[x][1]
        if red > blue:
            balance = blue/red
            protectedAttributeCountDict[x]["balance"] = balance
        else:
            balance = red/blue
            protectedAttributeCountDict[x]["balance"] = balance
    return protectedAttributeCountDict

def calculate_Fairness(G, communitiesList, attribute):
    
    protectedAttributeCountDict = initialize_community_attribute_Counter(communitiesList)

    protectedAttributeCountDict = count_protected_attributes_frequency(G, communitiesList, protectedAttributeCountDict, attribute)
        
    protectedAttributeCountDict = calculate_community_balance(protectedAttributeCountDict)

    return protectedAttributeCountDict

In [5]:
def most_frequent(list):
    return max(set(list), key=list.count)

In [100]:
# Execution Time 2mins
gplus_graph = nx.read_edgelist("{}gplus/gplus_combined.txt".format(path), nodetype=str, delimiter=" ", create_using=nx.DiGraph())

In [101]:
getProtectedAttributesGPlus(gplus_graph)

101560853443212199687.egofeat
109327480479767108490.egofeat
116899029375914044550.egofeat
{'114985346359714431656': 1, '111065108889012087599': 1, '113204882497955654314': 1, '116860750964767060846': 1, '109870053628419941069': 0, '108249232416813189685': 1, '111193388731102401849': 1, '111737859526639530840': 0, '112299972688047529628': 1, '102945758979783986480': 1, '100710846514306801296': 1, '106335292785856130691': 1, '107117483540235115863': 1, '114082129627545059603': 0, '103871883270165000246': 0, '115360979797396777969': 1, '115516333681138986628': 0, '106935269738929891479': 0, '114362374755134054810': 1, '101682840338947184385': 1, '106318643549045694881': 0, '100411393388776865339': 1, '114471843543990290947': 1, '112661262066626464523': 1, '101872015774361781592': 1, '101962829696981365403': 1, '101817213052779787512': 0, '101333593083741950617': 0, '110286587261352351537': 0, '117115495439924792722': 1, '111539653677669889601': 0, '106553543092892703402': 1, '115788203615

In [104]:
largestStronglyCCGplus = max(nx.strongly_connected_components(gplus_graph), key=len)
largestStronglyCCGplusGraph = gplus_graph.subgraph(largestStronglyCCGplus)

In [105]:
louvain_communities_Gplus = nx.algorithms.community.louvain_communities(largestStronglyCCGplusGraph)

KeyboardInterrupt: 

In [6]:
# Twitch gamers Graph
twitchGamers_graph = nx.read_edgelist("{}twitch_gamers/large_twitch_edges.csv".format(path), nodetype=int, delimiter=",")

In [7]:
# Twitch gamers features
twitchGamers_features = pd.read_csv("{}twitch_gamers/large_twitch_features.csv".format(path))

In [8]:
getProtectedAttributesTwitch(twitchGamers_graph, twitchGamers_features)

In [9]:
twitchGamersCC = max(nx.connected_components(twitchGamers_graph), key=len)
twitchGamersCCGraph = twitchGamers_graph.subgraph(twitchGamersCC)

In [10]:
# Εxecution time 5mins
louvain_communities_twitch = nx.algorithms.community.louvain_communities(twitchGamersCCGraph)

In [18]:
community_counter = 0
with open("output/twitchLouvain.txt", "w") as f:
    for community in louvain_communities_twitch:
        for node in community:
            f.write("{}\t{}\n".format(node, community_counter))
        community_counter += 1

In [22]:
louvain_gender_count = {}
louvain_gender_count = calculate_Fairness(twitchGamers_graph, louvain_communities_twitch, "mature")
louvain_gender_count

{'Community_0': {0: 3486, 1: 1523, 'balance': 0.4368904188181297},
 'Community_1': {0: 357, 1: 186, 'balance': 0.5210084033613446},
 'Community_2': {0: 10551, 1: 6717, 'balance': 0.6366221211259596},
 'Community_3': {0: 3754, 1: 1288, 'balance': 0.34310069259456577},
 'Community_4': {0: 237, 1: 266, 'balance': 0.8909774436090225},
 'Community_5': {0: 478, 1: 230, 'balance': 0.4811715481171548},
 'Community_6': {0: 189, 1: 153, 'balance': 0.8095238095238095},
 'Community_7': {0: 6981, 1: 10957, 'balance': 0.6371269508077029},
 'Community_8': {0: 556, 1: 173, 'balance': 0.31115107913669066},
 'Community_9': {0: 19502, 1: 16437, 'balance': 0.8428366321402933},
 'Community_10': {0: 3311, 1: 5490, 'balance': 0.6030965391621129},
 'Community_11': {0: 3446, 1: 1177, 'balance': 0.3415554265815438},
 'Community_12': {0: 1419, 1: 792, 'balance': 0.5581395348837209},
 'Community_13': {0: 910, 1: 254, 'balance': 0.27912087912087913},
 'Community_14': {0: 12945, 1: 20515, 'balance': 0.6310017060687

In [19]:
# 12m execution time
twitchGamersCCGraphComms = nx.algorithms.community.label_propagation_communities(twitchGamersCCGraph)

In [22]:
#maybe get the top 5 communities?
twitchLabelPropagationList = list(twitchGamersCCGraphComms)

In [25]:
community_counter = 0
with open("output/twitchLabelPropagation.txt", "w") as f:
    for community in twitchLabelPropagationList:
        for node in community:
            f.write("{}\t{}\n".format(node, community_counter))
        community_counter += 1

In [15]:
genderCountTwitch = {}
genderCountTwitch = calculate_Fairness(twitchGamers_graph, twitchLabelPropagationList, "mature")

In [16]:
genderCountTwitch

{'Community_0': {0: 83131, 1: 76960, 'balance': 0.9257677641313108},
 'Community_1': {0: 1563, 1: 1017, 'balance': 0.6506717850287908},
 'Community_2': {0: 2366, 1: 777, 'balance': 0.32840236686390534},
 'Community_3': {0: 473, 1: 135, 'balance': 0.2854122621564482},
 'Community_4': {0: 2, 1: 0, 'balance': 0.0},
 'Community_5': {0: 580, 1: 53, 'balance': 0.09137931034482759},
 'Community_6': {0: 854, 1: 44, 'balance': 0.05152224824355972},
 'Community_7': {0: 1, 1: 1, 'balance': 1.0},
 'Community_8': {0: 0, 1: 2, 'balance': 0.0},
 'Community_9': {0: 2, 1: 0, 'balance': 0.0},
 'Community_10': {0: 1, 1: 1, 'balance': 1.0},
 'Community_11': {0: 0, 1: 2, 'balance': 0.0},
 'Community_12': {0: 2, 1: 0, 'balance': 0.0},
 'Community_13': {0: 6, 1: 0, 'balance': 0.0},
 'Community_14': {0: 2, 1: 0, 'balance': 0.0},
 'Community_15': {0: 4, 1: 0, 'balance': 0.0},
 'Community_16': {0: 2, 1: 0, 'balance': 0.0},
 'Community_17': {0: 2, 1: 1, 'balance': 0.5},
 'Community_18': {0: 1, 1: 1, 'balance': 1

In [17]:
# 2m 31.6s execution time
twitchKernighan = nx.community.kernighan_lin_bisection(twitchGamersCCGraph, weight="weight")

In [19]:
genderCounttwitchKernighan = {}
genderCounttwitchKernighan = calculate_Fairness(twitchGamers_graph, twitchKernighan, "mature")

In [21]:
genderCounttwitchKernighan

{'Community_0': {0: 50786, 1: 33271, 'balance': 0.6551214901744575},
 'Community_1': {0: 38295, 1: 45762, 'balance': 0.836829684017307}}

In [26]:
# Deezer europe graph
deezer_graph = nx.read_edgelist("{}deezer_europe/deezer_europe/deezer_europe_edges.csv".format(path), nodetype=int, delimiter=",")

In [27]:
# Read Deezer genders
deezer_gendersDf = pd.read_csv("{}/deezer_europe/deezer_europe/deezer_europe_target.csv".format(path))

In [10]:
getProtectedAttributesDeezer(deezer_graph, deezer_gendersDf)

In [28]:
deezerLargestCC = max(nx.connected_components(deezer_graph), key=len)
deezerLargestCCGraph = deezer_graph.subgraph(deezerLargestCC)

In [29]:
deezerLabelPropagationComms = nx.algorithms.community.label_propagation_communities(deezerLargestCCGraph)

In [30]:
deezerLabelPropagationList = list(deezerLabelPropagationComms)

In [31]:
community_counter = 0
with open("output/deezerLabelPropagation.txt", "w") as f:
    for community in deezerLabelPropagationList:
        for node in community:
            f.write("{}\t{}\n".format(node, community_counter))
        community_counter += 1

In [32]:
louvain_communities_deezer = nx.community.louvain_communities(deezerLargestCCGraph)

In [34]:
community_counter = 0
with open("output/deezerLouvain.txt", "w") as f:
    for community in louvain_communities_deezer:
        for node in community:
            f.write("{}\t{}\n".format(node, community_counter))
        community_counter += 1

In [18]:
# Execution time 2m 26.1s
pokec_graph = nx.read_edgelist("{}/pokec/soc-pokec-relationships.txt".format(path), nodetype=int, delimiter="\t", create_using=nx.Graph())

In [14]:
pokec_features = pd.read_csv("{}/pokec/soc-pokec-profiles.txt".format(path), delimiter="\t", header=None, usecols=[0,3])

In [15]:
getProtectedAttributesPokec(pokec_graph, pokec_features)

In [16]:
largestStronglyCCPokec = max(nx.strongly_connected_components(pokec_graph), key=len)
largestStronglyCCPokecGraph = pokec_graph.subgraph(largestStronglyCCPokec)

In [17]:
labelPropagationPokec = nx.algorithms.community.label_propagation_communities(largestStronglyCCPokecGraph)

NetworkXNotImplemented: not implemented for directed type

In [35]:
facebook_graph_all = nx.read_edgelist("{}facebook/facebook/facebook_combined.txt".format(path), nodetype=int, delimiter=" ")

In [36]:
getProtectedAttributesFacebook(facebook_graph_all)

In [37]:
facebookLargestCC = max(nx.connected_components(facebook_graph_all), key=len)
facebookLargestCCGraph = facebook_graph_all.subgraph(facebookLargestCC)

In [38]:
facebookComms = nx.community.label_propagation_communities(facebookLargestCCGraph)

In [39]:
facebookLabelPropagationList = list(facebookComms)

In [43]:
community_counter = 0
with open("output/facebookLabelPropagation.txt", "w") as f:
    for community in facebookLabelPropagationList:
        for node in community:
            f.write("{}\t{}\n".format(node, community_counter))
        community_counter += 1

In [9]:
genderCountFacebook = {}
genderCountFacebook = calculate_Fairness(facebook_graph_all, facebookLabelPropagationList, 'gender')

In [10]:
genderCountFacebook

{'Community_0': {0: 122, 1: 76, 'balance': 0.6229508196721312},
 'Community_1': {0: 22, 1: 14, 'balance': 0.6363636363636364},
 'Community_2': {0: 8, 1: 2, 'balance': 0.25},
 'Community_3': {0: 4, 1: 4, 'balance': 1.0},
 'Community_4': {0: 5, 1: 3, 'balance': 0.6},
 'Community_5': {0: 23, 1: 11, 'balance': 0.4782608695652174},
 'Community_6': {0: 2, 1: 0, 'balance': 0.0},
 'Community_7': {0: 168, 1: 47, 'balance': 0.27976190476190477},
 'Community_8': {0: 8, 1: 8, 'balance': 1.0},
 'Community_9': {0: 2, 1: 1, 'balance': 0.5},
 'Community_10': {0: 1, 1: 2, 'balance': 0.5},
 'Community_11': {0: 674, 1: 356, 'balance': 0.5281899109792285},
 'Community_12': {0: 4, 1: 2, 'balance': 0.5},
 'Community_13': {0: 5, 1: 2, 'balance': 0.4},
 'Community_14': {0: 1, 1: 2, 'balance': 0.5},
 'Community_15': {0: 3, 1: 0, 'balance': 0.0},
 'Community_16': {0: 383, 1: 369, 'balance': 0.9634464751958225},
 'Community_17': {0: 6, 1: 4, 'balance': 0.6666666666666666},
 'Community_18': {0: 0, 1: 2, 'balance'

In [41]:
louvain_communities_facebook = nx.community.louvain_communities(facebookLargestCCGraph)

In [44]:
community_counter = 0
with open("output/facebookLouvain.txt", "w") as f:
    for community in louvain_communities_facebook:
        for node in community:
            f.write("{}\t{}\n".format(node, community_counter))
        community_counter += 1

In [45]:
girvan_newman_facebook = nx.algorithms.community.girvan_newman(facebookLargestCCGraph)